In [23]:
from sklearn import metrics,datasets,semi_supervised,svm
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.semi_supervised import SelfTrainingClassifier


# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split the data into labeled and unlabeled sets
X_labeled, X_unlabeled, y_labeled,y_hide = train_test_split(X, y, test_size=0.5, random_state=42)
y_unlabeled = np.full(len(X_unlabeled), -1)



X_new=np.concatenate([X_labeled,X_unlabeled])
y_new=np.concatenate([y_labeled,y_unlabeled])




# Define pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', SelfTrainingClassifier(SVC(probability=True)))
])

# Define parameter grid for SVC
param_grid = {'clf__base_estimator__C': [0.1, 1, 10],
              'clf__base_estimator__gamma': [0.1, 0.01, 0.001],
              'clf__base_estimator__random_state':[123],
              'clf__base_estimator__degree':[3,4,5,6]}

# Perform grid search
grid_search = GridSearchCV(pipeline, param_grid, cv=5)
grid_search.fit(X_new, y_new)
Y_pred=grid_search.predict(X_unlabeled)
metrics.accuracy_score(Y_pred,y_hide)

0.9866666666666667

In [29]:
pd.DataFrame(grid_search.cv_results_).sort_values(['mean_test_score'],ascending=False)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__base_estimator__C,param_clf__base_estimator__degree,param_clf__base_estimator__gamma,param_clf__base_estimator__random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
33,0.018308,0.003101,0.000000,0.000000,10,6,0.1,123,"{'clf__base_estimator__C': 10, 'clf__base_esti...",0.433333,0.433333,0.500000,0.500000,0.466667,0.466667,0.029814,1
30,0.017919,0.002767,0.000000,0.000000,10,5,0.1,123,"{'clf__base_estimator__C': 10, 'clf__base_esti...",0.433333,0.433333,0.500000,0.500000,0.466667,0.466667,0.029814,1
27,0.017546,0.001698,0.000000,0.000000,10,4,0.1,123,"{'clf__base_estimator__C': 10, 'clf__base_esti...",0.433333,0.433333,0.500000,0.500000,0.466667,0.466667,0.029814,1
24,0.016896,0.000876,0.001164,0.002328,10,3,0.1,123,"{'clf__base_estimator__C': 10, 'clf__base_esti...",0.433333,0.433333,0.500000,0.500000,0.466667,0.466667,0.029814,1
18,0.030886,0.006599,0.000102,0.000204,1,5,0.1,123,"{'clf__base_estimator__C': 1, 'clf__base_estim...",0.466667,0.466667,0.466667,0.500000,0.433333,0.466667,0.021082,5
15,0.027782,0.003816,0.000753,0.001011,1,4,0.1,123,"{'clf__base_estimator__C': 1, 'clf__base_estim...",0.466667,0.466667,0.466667,0.500000,0.433333,0.466667,0.021082,5
34,0.019188,0.002310,0.001196,0.001759,10,6,0.01,123,"{'clf__base_estimator__C': 10, 'clf__base_esti...",0.466667,0.433333,0.500000,0.500000,0.433333,0.466667,0.029814,5
31,0.020331,0.004054,0.002888,0.003575,10,5,0.01,123,"{'clf__base_estimator__C': 10, 'clf__base_esti...",0.466667,0.433333,0.500000,0.500000,0.433333,0.466667,0.029814,5
28,0.021675,0.004558,0.000249,0.000310,10,4,0.01,123,"{'clf__base_estimator__C': 10, 'clf__base_esti...",0.466667,0.433333,0.500000,0.500000,0.433333,0.466667,0.029814,5
25,0.018530,0.003875,0.003113,0.003334,10,3,0.01,123,"{'clf__base_estimator__C': 10, 'clf__base_esti...",0.466667,0.433333,0.500000,0.500000,0.433333,0.466667,0.029814,5


In [79]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import LabelPropagation
import numpy as np
ubah=0
confi=1
lb_confi=0.5


# Load iris dataset
iris = load_iris()
X_labeled, X_unlabeled, y_labeled, _ = train_test_split(iris.data, iris.target, test_size=0.5, stratify=iris.target)

# Create initial classifiers
clf1 = RandomForestClassifier()
clf2 = RandomForestClassifier()

# Train initial classifiers on labeled data
clf1.fit(X_labeled, y_labeled)
clf2.fit(X_labeled, y_labeled)

# Start co-training process
for i in range(100):  # Number of iterations
    try:
        
        if len(X_unlabeled)==0:
            break
        
        if confi <= lb_confi:
                confi = lb_confi
        if ubah>5:
                break
        if ubah == 3:
            confi = confi - 0.05
                
                
        # Predict labels for unlabeled data
        print(i)
        pred1 = clf1.predict(X_unlabeled)
        pred2 = clf2.predict(X_unlabeled)

        # Select most confident predictions
        confidence1 = np.max(clf1.predict_proba(X_unlabeled), axis=1)
        confidence2 = np.max(clf2.predict_proba(X_unlabeled), axis=1)
        high_confidence_indices1 = np.where(confidence1 >= confi)[0]
        high_confidence_indices2 = np.where(confidence2 >= confi)[0]
        selected_indices = np.intersect1d(high_confidence_indices1, high_confidence_indices2)
        print(len(selected_indices), len(X_unlabeled))
        print('confi==>>>>',confi)
        
        

        

        if len(selected_indices) > 0:
            # Predictions made by both classifiers for the selected instances
            pred1_selected = pred1[selected_indices]
            pred2_selected = pred2[selected_indices]

            # Check if predictions made by both classifiers are the same
            same_predictions_indices = np.where(pred1_selected == pred2_selected)[0]
            print(pred1_selected)
            print(pred2_selected)

            # Add instances with same predictions to the labeled data
            if len(same_predictions_indices) > 0:
                X_labeled = np.vstack((X_labeled, X_unlabeled[selected_indices[same_predictions_indices]]))
                y_labeled = np.hstack((y_labeled, pred1[selected_indices[same_predictions_indices]]))

                # Remove added instances from the unlabeled data
                X_unlabeled = np.delete(X_unlabeled, selected_indices[same_predictions_indices], axis=0)
        else:
            print(0)

        
            
            if len(selected_indices) > 0:
                print('ubah')
                X_labeled = np.vstack((X_labeled, X_unlabeled[selected_indices]))
                y_labeled = np.hstack((y_labeled, pred1[selected_indices]))  # or use pred2

                X_unlabeled = np.delete(X_unlabeled, selected_indices, axis=0)
        if len(selected_indices) == 0:
            ubah = ubah + 1
        else:
            ubah = 0
        print(ubah)
        print()

        # Train classifiers on expanded labeled data
        clf1.fit(X_labeled, y_labeled)
        clf2.fit(X_labeled, y_labeled)
    except Exception as e:
        print("Error occurred:", e)
        break
        # Return error or handle it accordingly

    


# Evaluate performance on test set
X_test, y_test = iris.data, iris.target
accuracy1 = clf1.score(X_test, y_test)
accuracy2 = clf2.score(X_test, y_test)
print("Classifier 1 accuracy:", accuracy1)
print("Classifier 2 accuracy:", accuracy2)
print(len(X_unlabeled))

0
33 75
confi==>>>> 1
[0 0 0 2 2 2 0 2 0 0 2 2 2 0 0 0 0 2 0 0 1 0 2 1 2 2 2 0 2 2 2 2 0]
[0 0 0 2 2 2 0 2 0 0 2 2 2 0 0 0 0 2 0 0 1 0 2 1 2 2 2 0 2 2 2 2 0]
0

1
8 42
confi==>>>> 1
[1 2 1 2 0 0 0 0]
[1 2 1 2 0 0 0 0]
0

2
4 34
confi==>>>> 1
[1 1 1 0]
[1 1 1 0]
0

3
0 30
confi==>>>> 1
0
1

4
0 30
confi==>>>> 1
0
2

5
0 30
confi==>>>> 1
0
3

6
13 30
confi==>>>> 0.95
[2 1 1 2 1 2 2 1 1 1 1 2 2]
[2 1 1 2 1 2 2 1 1 1 1 2 2]
0

7
3 17
confi==>>>> 0.95
[1 2 1]
[1 2 1]
0

8
1 14
confi==>>>> 0.95
[0]
[0]
0

9
0 13
confi==>>>> 0.95
0
1

10
0 13
confi==>>>> 0.95
0
2

11
0 13
confi==>>>> 0.95
0
3

12
4 13
confi==>>>> 0.8999999999999999
[0 0 1 1]
[0 0 1 1]
0

13
3 9
confi==>>>> 0.8999999999999999
[1 0 0]
[1 0 0]
0

14
1 6
confi==>>>> 0.8999999999999999
[1]
[1]
0

15
0 5
confi==>>>> 0.8999999999999999
0
1

16
0 5
confi==>>>> 0.8999999999999999
0
2

17
0 5
confi==>>>> 0.8999999999999999
0
3

18
1 5
confi==>>>> 0.8499999999999999
[1]
[1]
0

19
0 4
confi==>>>> 0.8499999999999999
0
1

20
1 4
confi==>>>

In [42]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.semi_supervised import LabelPropagation
import numpy as np

# Load iris dataset
iris = load_iris()
X_labeled, X_unlabeled, y_labeled, _ = train_test_split(iris.data, iris.target, test_size=0.5, stratify=iris.target)

# Create initial classifiers
clf1 = RandomForestClassifier()
clf2 = RandomForestClassifier()

# Train initial classifiers on labeled data
clf1.fit(X_labeled, y_labeled)
clf2.fit(X_labeled, y_labeled)


RandomForestClassifier()

In [44]:
pred1 = clf1.predict(X_unlabeled)
pred2 = clf2.predict(X_unlabeled)
print('pred1,pred2')
print(pred1,'\n\n',pred2)

pred1,pred2
[1 0 2 0 2 2 1 1 2 0 2 0 2 2 1 2 1 2 1 2 2 2 0 0 1 0 2 1 2 1 2 1 2 0 2 0 2
 1 0 1 2 1 1 2 2 1 0 0 2 0 2 0 2 2 0 1 1 0 2 1 1 0 0 0 0 1 0 1 0 0 1 1 2 0
 0] 

 [1 0 2 0 2 2 1 1 2 0 2 0 2 2 1 2 1 2 1 2 2 2 0 0 1 0 2 1 2 1 2 1 2 0 2 0 2
 1 0 1 2 1 1 2 2 1 0 0 2 0 2 0 2 2 0 1 1 0 2 1 1 0 0 0 0 1 0 1 0 0 1 1 2 0
 0]


In [45]:
# Select most confident predictions
confidence1 = np.max(clf1.predict_proba(X_unlabeled), axis=1)
confidence2 = np.max(clf2.predict_proba(X_unlabeled), axis=1)
print('confidence1,confidence2')
print(confidence1,'\n\n',confidence2)

confidence1,confidence2
[0.99 1.   1.   1.   0.6  0.98 0.97 0.81 1.   1.   1.   1.   1.   0.97
 1.   1.   0.94 0.75 0.99 0.94 0.99 0.97 1.   1.   0.77 1.   1.   0.92
 1.   1.   0.77 1.   1.   1.   0.98 1.   0.99 0.89 1.   0.98 1.   0.99
 0.99 1.   1.   1.   0.96 1.   0.56 1.   0.73 0.91 1.   1.   1.   0.99
 0.99 1.   0.9  0.97 0.98 0.96 1.   1.   1.   1.   0.99 0.94 1.   1.
 0.99 0.99 0.8  1.   0.94] 

 [1.   1.   1.   1.   0.68 1.   0.97 0.84 1.   1.   0.99 1.   1.   1.
 0.99 1.   0.89 0.74 0.98 0.93 0.93 0.97 1.   1.   0.71 1.   1.   0.98
 1.   0.98 0.8  0.97 1.   1.   1.   1.   0.99 0.85 1.   0.99 0.96 0.99
 1.   1.   0.97 0.97 0.96 1.   0.56 1.   0.76 0.93 1.   0.98 1.   0.99
 0.99 1.   0.89 1.   0.97 0.95 1.   1.   1.   0.97 1.   0.99 1.   1.
 0.99 1.   0.84 1.   0.9 ]


In [46]:
high_confidence_indices1 = np.where(confidence1 > 0.8)[0]
high_confidence_indices2 = np.where(confidence2 > 0.8)[0]
print('high_confidence_indices1,high_confidence_indices2')
print(high_confidence_indices1,'\n\n',high_confidence_indices2)

high_confidence_indices1,high_confidence_indices2
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 25 26
 27 28 29 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 49 51 52 53
 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 73 74] 

 [ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 25 26
 27 28 29 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 49 51 52 53
 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74]


In [47]:
selected_indices = np.intersect1d(high_confidence_indices1, high_confidence_indices2)
print('selected_indices')
print(selected_indices)

selected_indices
[ 0  1  2  3  5  6  7  8  9 10 11 12 13 14 15 16 18 19 20 21 22 23 25 26
 27 28 29 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 49 51 52 53
 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 73 74]


In [49]:
if len(selected_indices) > 0:
    X_labeled = np.vstack((X_labeled, X_unlabeled[selected_indices]))
    print('X_labeled')
    print(X_labeled)
    
    
    y_labeled = np.hstack((y_labeled, pred1[selected_indices]))  # or use pred2
    print('\ny_labeled')
    print(y_labeled)

    X_unlabeled = np.delete(X_unlabeled, selected_indices, axis=0)
    print('\nX_unlabeled')
    print(X_unlabeled)

X_labeled
[[4.9 3.1 1.5 0.2]
 [5.  2.3 3.3 1. ]
 [5.5 2.3 4.  1.3]
 [6.6 3.  4.4 1.4]
 [6.8 3.  5.5 2.1]
 [5.2 3.5 1.5 0.2]
 [6.  2.2 5.  1.5]
 [6.5 3.2 5.1 2. ]
 [7.2 3.6 6.1 2.5]
 [6.3 2.8 5.1 1.5]
 [6.9 3.1 4.9 1.5]
 [6.1 2.9 4.7 1.4]
 [5.6 3.  4.1 1.3]
 [6.3 3.3 4.7 1.6]
 [6.  2.2 4.  1. ]
 [5.  3.4 1.6 0.4]
 [6.3 2.5 5.  1.9]
 [7.7 3.  6.1 2.3]
 [6.3 3.3 6.  2.5]
 [4.6 3.4 1.4 0.3]
 [5.8 2.7 4.1 1. ]
 [5.  3.4 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [6.  3.4 4.5 1.6]
 [5.7 2.8 4.1 1.3]
 [6.9 3.1 5.4 2.1]
 [4.9 3.6 1.4 0.1]
 [6.7 3.1 4.4 1.4]
 [6.2 2.9 4.3 1.3]
 [4.9 2.5 4.5 1.7]
 [4.6 3.1 1.5 0.2]
 [6.1 2.8 4.  1.3]
 [4.4 3.  1.3 0.2]
 [5.5 2.6 4.4 1.2]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]
 [7.6 3.  6.6 2.1]
 [5.7 2.6 3.5 1. ]
 [4.9 3.  1.4 0.2]
 [5.9 3.2 4.8 1.8]
 [4.8 3.  1.4 0.3]
 [7.2 3.2 6.  1.8]
 [5.8 2.8 5.1 2.4]
 [5.  3.5 1.3 0.3]
 [6.2 2.2 4.5 1.5]
 [7.7 2.8 6.7 2. ]
 [6.3 2.5 4.9 1.5]
 [4.9 3.1 1.5 0.1]
 [4.8 3.  1.4 0.1]
 [6.2 2.8 4.8 1.8]
 [5.4 3.4 1.7 0.2]
 [5.  3.2 1.2 0.2]
 [

In [41]:

# Train classifiers on expanded labeled data
clf1.fit(X_labeled, y_labeled)
clf2.fit(X_labeled, y_labeled)

# Evaluate performance on test set
X_test, y_test = iris.data, iris.target
accuracy1 = clf1.score(X_test, y_test)
accuracy2 = clf2.score(X_test, y_test)
print("Classifier 1 accuracy:", accuracy1)
print("Classifier 2 accuracy:", accuracy2)


pred1,pred2
[2 2 1 2 2] [2 2 1 1 2]
confidence1,confidence2
[0.74 0.71 0.66 0.52 0.79] [0.77 0.69 0.73 0.51 0.79]
high_confidence_indices1,high_confidence_indices2
[] []
selected_indices
[]
Classifier 1 accuracy: 0.9933333333333333
Classifier 2 accuracy: 0.9933333333333333
